# Selection algorithms

In [1]:
# Community simulator package
from IPython.display import Image
from community_simulator import *
from community_simulator.usertools import *
from community_simulator.visualization import *
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.backends import backend_pdf as bpdf
import numpy as np
import scipy as sp
colors = sns.color_palette()
%matplotlib inline

# Community selection package
from community_selection import *
from community_selection.A_experiment_functions import *
from community_selection.B_community_phenotypes import *
from community_selection.C_selection_algorithms import *
from community_selection.D_migration_algorithms import *

In [2]:
# Make dynanmics by default we will use the microbial consumer resource model
def dNdt(N,R,params):
    return MakeConsumerDynamics(assumptions)(N,R,params)
def dRdt(N,R,params):
    return MakeResourceDynamics(assumptions)(N,R,params)
dynamics = [dNdt,dRdt]

# Global parameters
## Default parameters from community-simulator
## !!!Don't touch this dictionary!!!
assumptions = a_default.copy() # Start with default parameters

## Update parameters for community-selection
assumptions.update({
    'SA': 600*np.ones(3), #Number of species in each specialist family (here, 3 families of 60 species)
    'MA': 30*np.ones(3), #Number of resources in each class
    'Sgen': 300, #Number of generalist species (unbiased sampling over alll resource classes)
    "n_wells": 10,
    "m": 0, # Mortality
    "scale": 10**6,  #scale is a conversion factor specifying the number of individual microbial cells present when N = 1.
    "sigma" : 1, # Standard deviation for drawing specifc speices/interaction function
    "alpha": 1, # Scaling factor between species- and interaction-specific function variances
    "l": 0, # Set leakage function to 0 to switch off cross-feeding
    "response": "type III",
    "sigma_max": 5,
    'R0_food': 1000, # Total amount of supplied food
    "rich_medium": True, # Number of food types passed to R0
    "binary_threshold": 1,  
    # The parameters below will be passed to params_simulation
    "n_propagation": 1, # Length of propagation, or hours within a growth cycle
    "n_transfer": 10, # Number of total transfer, or number of passage
    "n_transfer_selection": 5, # Number of transfer implementing seleciton regimes
    "dilution": 1/1000, # Dilution factor at every transfer
    "n_inoc": 10**6,  #Number of cells sampled from the regional species at start
    "selected_function": "f1_additive"
})

# Prepare experiment setup in this universe
seed_temp = 1
params, params_simulation = prepare_experiment(assumptions, seed = seed_temp)

In [3]:
data_directory = "data/test/"

list_phenotypes = ["f7_resident_growth_community", "f8_resident_growth_environment"]
#list_phenotypes = ["f1_additive", "f2_interaction", "f3_additive_binary", "f4_interaction_binary", "f5_invader_growth", "f6_resident_growth"]

list_algorithms = ["simple_screening"]
#list_algorithms = ["simple_screening", "directed_selection_migration", "pair_top_communities", "multiple_pair_top", "Blouin2015", "Mueller2019", "Panke_Buisse2015", "Swenson2000a", "Swenson2000b", "Williams2007a", "Williams2007b", "Wright2019", "coalescence", "migration", "resource", "bottleneck", "knock_out", "knock_in"]


for j in range(len(list_phenotypes)):
    # Parameters for simulation
    params_simulation.update({"selected_function": list_phenotypes[j]}) # selected function

    # Make the list of algorithms
    algorithms = make_algorithms(params_simulation)

    # Simulation
    for i in range(len(list_algorithms)):
        simulate_community(
            assumptions = assumptions,
            params = params,
            dynamics = dynamics,
            params_simulation = params_simulation,
            params_algorithm = algorithms[algorithms["algorithm_name"] == list_algorithms[i]],
            write_composition = True,
            file_name = data_directory + "SP" + str(seed_temp) + "-" + list_algorithms[i],
            assembly_type = str(list_algorithms[i]),
        )


Algorithm: simple_screening


 transfer           community_phenotype selection_algorithm migration_algorithm
        1  f7_resident_growth_community        no_selection        no_migration
        2  f7_resident_growth_community        no_selection        no_migration
        3  f7_resident_growth_community        no_selection        no_migration
        4  f7_resident_growth_community        no_selection        no_migration
        5  f7_resident_growth_community        no_selection        no_migration
        6  f7_resident_growth_community        no_selection        no_migration
        7  f7_resident_growth_community        no_selection        no_migration
        8  f7_resident_growth_community        no_selection        no_migration
        9  f7_resident_growth_community        no_selection        no_migration
       10  f7_resident_growth_community        no_selection        no_migration

Generating initial plate

Adding attributes that are essential to the community function

In [6]:
# Set seeds
np.random.seed(2)

# Make initial state
init_state = MakeInitialState(assumptions)

# Make plate
plate = Community(init_state, dynamics, params, scale = assumptions["scale"], parallel = True) 

# Update the community composition by sampling from the pool
print("\nGenerating initial plate")
plate.N = sample_from_pool(plate.N, scale = assumptions["scale"], inocula = params_simulation["n_inoc"])

# Update the supplied resource if assumptions["rich_medium"]
if assumptions["rich_medium"]:
    plate.R = make_rich_medium(plate.R, assumptions)
    plate.R0 = make_rich_medium(plate.R, assumptions) # R0 for refreshing media on passaging if "refresh_resoruce" is turned on

# Add the attributes that are essential to the function measurement to the plate objects 
print("\nAdding attributes that are essential to the community function to the plate object")
plate = add_community_function(plate, dynamics, assumptions, params_simulation)



Generating initial plate

Adding attributes that are essential to the community function to the plate object
Sampling invader (resident) community

Stabilizing the invader (resident) community. Passage for 5 transfer. The plate has  10  wells.
Passaging invader (resident) community. Transfer 1
Passaging invader (resident) community. Transfer 2
Passaging invader (resident) community. Transfer 3
Passaging invader (resident) community. Transfer 4
Passaging invader (resident) community. Transfer 5
Finished passaging the invader (resident) community.
The community has 99 species
The community has initial biomass 1 and reaches total biomass 1000.1449943518375


In [14]:
plate.resident_plate_t1

W0         W1         W2         W3         W4         W5  \
F0  S0      0.000000   0.000000   0.000000   0.000000   0.000000   0.000000   
    S1      0.000000   0.000000   0.000000   0.000000   0.000000   0.000000   
    S2      0.000000   0.000000   0.000000   0.000000   0.000000   0.000000   
    S3      0.000000   0.000000   0.000000   0.000000   0.000000   0.000000   
    S4      0.000000   0.000000   0.000000   0.000000   0.000000   0.000000   
    S5      0.000000   0.000000   0.000000   0.000000   0.000000   0.000000   
    S6      0.000000   0.000000   0.000000   0.000000   0.000000   0.000000   
    S7      0.000000   0.000000   0.000000   0.000000   0.000000   0.000000   
    S8      0.000000   0.000000   0.000000   0.000000   0.000000   0.000000   
    S9      0.000000   0.000000   0.000000   0.000000   0.000000   0.000000   
    S10     0.000000   0.000000   0.000000   0.000000   0.000000   0.000000   
    S11     0.000000   0.000000   0.000000   0.000000   0.000000   0.000000   
    S12    23.504728  23.504728  23.504728  23.504728  23.504728  23.504728   
    S13     0.000000   0.000000   0.000000   0.000000   0.000000   0.000000   
    S14     0.000000   0.000000   0.000000   0.000000   0.000000   0.000000   
    S15     0.000000   0.000000   0.000000   0.000000   0.000000   0.000000   
    S16     0.000000   0.000000   0.000000   0.000000   0.000000   0.000000   
    S17     0.000000   0.000000   0.000000   0.000000   0.000000   0.000000   
    S18     0.000000   0.000000   0.000000   0.000000   0.000000   0.000000   
    S19     0.000000   0.000000   0.000000   0.000000   0.000000   0.000000   
    S20     0.000000   0.000000   0.000000   0.000000   0.000000   0.000000   
    S21     0.000000   0.000000   0.000000   0.000000   0.000000   0.000000   
    S22     0.000000   0.000000   0.000000   0.000000   0.000000   0.000000   
    S23     0.000000   0.000000   0.000000   0.000000   0.000000   0.000000   
    S24     0.000000   0.000000   0.000000   0.000000   0.000000   0.000000   
    S25     0.000000   0.000000   0.000000   0.000000   0.000000   0.000000   
    S26     0.000280   0.000280   0.000280   0.000280   0.000280   0.000280   
    S27     0.000000   0.000000   0.000000   0.000000   0.000000   0.000000   
    S28     0.000000   0.000000   0.000000   0.000000   0.000000   0.000000   
    S29     0.237965   0.237965   0.237965   0.237965   0.237965   0.237965   
...              ...        ...        ...        ...        ...        ...   
GEN S2070   0.000000   0.000000   0.000000   0.000000   0.000000   0.000000   
    S2071   0.000000   0.000000   0.000000   0.000000   0.000000   0.000000   
    S2072   0.000000   0.000000   0.000000   0.000000   0.000000   0.000000   
    S2073   0.000000   0.000000   0.000000   0.000000   0.000000   0.000000   
    S2074  34.631580  34.631580  34.631580  34.631580  34.631580  34.631580   
    S2075   0.000000   0.000000   0.000000   0.000000   0.000000   0.000000   
    S2076   0.000000   0.000000   0.000000   0.000000   0.000000   0.000000   
    S2077   0.000000   0.000000   0.000000   0.000000   0.000000   0.000000   
    S2078   0.000000   0.000000   0.000000   0.000000   0.000000   0.000000   
    S2079   0.000000   0.000000   0.000000   0.000000   0.000000   0.000000   
    S2080   0.000000   0.000000   0.000000   0.000000   0.000000   0.000000   
    S2081   0.000000   0.000000   0.000000   0.000000   0.000000   0.000000   
    S2082   0.000000   0.000000   0.000000   0.000000   0.000000   0.000000   
    S2083   0.000000   0.000000   0.000000   0.000000   0.000000   0.000000   
    S2084   0.000000   0.000000   0.000000   0.000000   0.000000   0.000000   
    S2085   0.000000   0.000000   0.000000   0.000000   0.000000   0.000000   
    S2086   0.000000   0.000000   0.000000   0.000000   0.000000   0.000000   
    S2087   0.000000   0.000000   0.000000   0.000000   0.000000   0.000000   
    S2088   0.000000   0.000000   0.000000   0.000000   0.000000   0